In [2]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import json
import re

url = 'https://www.d20pfsrd.com/classes/'

class ArchetypeScraper:
    def __init__(self, url):
        self.url = url
        self.href = f'alternate-classes/ninja/ninja-tricks/'
        self.page = requests.get(url + self.href)
        self.soup = BeautifulSoup(self.page.content, 'html.parser')
        self.table = self.soup.find("table")
        self.href2_list = []



    def clean_text(self, text):
        # Replace Unicode right single quotation mark with an apostrophe
        cleaned_text = [line.replace('\u2019', "'").replace('\u2018', "'").replace('\u201c', '"').replace('\u201d', '"') for line in text.split('\n') if line.strip() and line.strip() != "."]
        return ' '.join(cleaned_text).strip()
    
    def remove_parenthesis(self, input_string):
        pattern = r"\(.*?pg\..*?\)"
        result = re.sub(pattern, '', input_string)
        result = re.sub(r'<.*?>', '', result)        

        return result

    def link_searcher(self, a_tags):
        for a in a_tags:
            # Check if the link contains "investigator-talents" in the href attribute
            if "paizo-ninja-tricks" in str(a.get('href', '')):
                # Extract everything within double quotes in the href attribute
                regex = r'href="(.*?)"'
                href_unclean = str(a)
                match = re.search(regex, href_unclean)

                if match:
                    self.href2 = match.group(1)
                    self.href2_list.append(self.href2)
                else:
                    continue
            else:
                continue    


        return self.href2_list

         


# What we want to do is to get the data once we see a <b> tag, use the <b> tag as the key section
# Since it's going to be the same each time we can manually set them (like in the items json file)
    def div_search(self, td_tags):
        
        result_list = []

        for td in td_tags:
            a_tags = td.find_all('a')
            data_dict = {}            
            href2_list = self.link_searcher(a_tags)
        
        
    def talent_info_search(self):
        all_data_dicts = []

        for url2 in self.href2_list:
            self.detail_page = requests.get(url2)
            self.detail_soup = BeautifulSoup(self.detail_page.content, 'html.parser')

            div_tags = self.detail_soup.find_all('div', class_="article-content")
            b_tags = self.detail_soup.find_all('b')

            for div in div_tags:
                p_tags = div.find_all('p')
                h_tag = div.find('h1')

                data_dict = {}
                data_dict[h_tag.get_text()] = {}

                for p_tag in p_tags:
                    # Split text at ':' to get key-value pairs
                    p_tag_clean = self.remove_parenthesis(p_tag.get_text())
                    parts = p_tag_clean.split(':')
                    if len(parts) > 1:
                        key = self.remove_parenthesis(parts[0].strip())
                        value = self.remove_parenthesis(parts[1].strip())
                        data_dict[h_tag.get_text()][key] = value

                all_data_dicts.append(data_dict)

        # Convert the list of dictionaries to a JSON string
        json_data = json.dumps(all_data_dicts, indent=2)

        with open('ninja.json', 'w', encoding='utf-8') as json_file:
            json.dump(all_data_dicts, json_file, indent=2, ensure_ascii=False)        

        # Print or return the JSON string as needed
        print(json_data)
        return json_data



    def main(self):        
        td_tags = self.soup.find_all('td')
        self.div_search(td_tags)
        self.talent_info_search()


# Instantiate the ArchetypeScraper class
scraper_instance = ArchetypeScraper(url)

# Call the get_class_info method
scraper_instance.main()

[
  {
    "Acceleration of Form (Su)": {
      "Benefit": "A ninja with this trick can spend 1 ki point as a standard action to gain the benefits of displacement and haste for 1 round per 2 ninja levels.",
      "Pathfinder Player Companion": "Chronicle of Legends \u00a9 2019, Paizo Inc.; Authors"
    }
  },
  {
    "Acrobatic Master (Su)": {
      "Benefit": "As a swift action, a ninja with this trick can focus her ki to grant her a +20 bonus on one Acrobatics check of her choice made before the start of her next turn. Using this ability expends 1 ki point from her ki pool.",
      "Pathfinder Roleplaying Game Ultimate Combat \u00a9 2011, Paizo Publishing, LLC; Authors": "Jason Bulmahn, Tim Hitchcock, Colin McComb, Rob McCreary, Jason Nelson, Stephen Radney-MacFarland, Sean K Reynolds, Owen K.C. Stephens, and Russ Taylor"
    }
  },
  {
    "Bleeding Attack* (Ex)": {
      "Benefit": "A ninja with this trick can cause living opponents to bleed by hitting them with a sneak attack. This